In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext memray

import resource

# constrain our own RAM for safety
resource.setrlimit(resource.RLIMIT_AS, (18 * 1024 * 1024 * 1024, resource.RLIM_INFINITY))

In [ ]:
import os
from pprint import pp

from evo.data_converters.common import create_evo_object_service_and_data_client
from evo.notebooks import ServiceManagerWidget

client_id = os.getenv("EVO_CLIENT_ID", "")
base_uri = os.getenv("EVO_BASE_URI", "")
discovery_url = os.getenv("EVO_DISCOVERY_URL", "")

manager = await ServiceManagerWidget.with_auth_code(
    client_id=client_id, base_uri=base_uri, discovery_url=discovery_url
).login()

object_service_client, data_client = create_evo_object_service_and_data_client(service_manager_widget=manager)

In [ ]:
from pathlib import Path

# test_file = Path.cwd() / "data" / "600K_triangles.obj.zip"
# test_file = Path.cwd() / "data" / "hongkong.obj.zip"
# test_file = Path.cwd() / "data" / "hongkong" / "level_5_0_0.obj"
# test_file = Path.cwd() / "data" / "powerplant" / "powerplant.obj"
test_file = Path.cwd() / "data" / "170917_StellarModel_Datapackage" / "170917_StellarModel.obj"

assert test_file.exists()

In [ ]:
%%prun
# %%memray_flamegraph --temporal --trace-python-allocators --leaks --native

import asyncio
import nest_asyncio
import pyarrow as pa
import pyarrow.parquet as pq
from uuid import UUID
from time import monotonic
from pathlib import Path
from unittest.mock import patch
from evo.data_converters.obj.importer.obj_to_evo import convert_obj

nest_asyncio.apply()


def fake_save_table(table: pa.Table):
    start = monotonic()
    if "x" in table.column_names:
        table_file = "/tmp/verts.parquet"
    elif "n0" in table.column_names:
        table_file = "/tmp/triangles.parquet"
    elif "count" in table.column_names:
        table_file = "/tmp/parts.parquet"
    else:
        print(table)
        raise NotImplementedError("Unknown table?")
    pq.write_table(table=table, where=table_file, version="2.4", compression="gzip")
    end = monotonic()
    print(f"Wrote {table_file} in {end - start}s")
    return dict(data=UUID("00000000-0000-0000-0000-000000000000"), length=123)


with patch("evo.objects.utils.data.ObjectDataClient.save_table", side_effect=fake_save_table):
    tags = {"TagName": "Tag value"}
    implementation = "tinyobj"
    objects_metadata = asyncio.run(
        convert_obj(
            filepath=test_file,
            implementation=implementation,
            epsg_code=32650,
            service_manager_widget=manager,
            tags=tags,
            upload_path=f"test-obj-{implementation}",
            publish_objects=False,
            overwrite_existing_objects=True,
        )
    )

print()
print("These objects have now been published:")

for metadata in objects_metadata:
    pp(metadata, indent=4)